In [1]:
import pandas as pd
import numpy as np
import dateutil.parser

In [2]:
from os.path import join as path, dirname

try:
    from IPython.core.display import HTML

    def pprint(df):
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(HTML(pd.DataFrame(df).to_html()))
except:
    def pprint(df):
        print(df)

In [3]:
df_books = pd.read_excel(path('data', 'BooksDB.xlsx'), index_col='idBook')
pprint(df_books)

,title,auther,publisher,publicationYear,ISBN,nPages,roomNumber,rowNumber,bookShelfNumber
idBook,,,,,,,,,
1,Texts from Denmark,Brian Jensen,Gyldendal,1837-10-22 00:00:00,0-4353-0966-8,915,1A,N101,1AA2
2,Foreigner Of Next Year,Alan Lawrence,Johns-Hoppe,1837-10-22 00:00:00,0-1387-1756-7,767,1A,N107,7AA3
3,Owl With Wings,Tim Cornish,Braun-Carroll,1864-03-10 00:00:00,0-7767-2473-8,683,1A,N101,1AA4
4,Defenders Of The South,Austin Duncan,Greenholt Group,1837-10-22 00:00:00,0-9600-5882-6,910,1A,N103,3AA2
5,Witches Of The Ancestors,Audrey Bailey,Buckridge-Dach,1861-11-12 00:00:00,0-9931-8518-5,565,1A,N105,5AA3
6,Robots And Doctors,Hannah Ross,Block-Bernhard,1853-04-04 00:00:00,0-7191-2309-7,346,1A,N104,4AA4
7,Pilots And Priests,Dorothy Hardacre,Feeney-Beatty,1837-10-22 00:00:00,0-6097-0325-0,667,2A,N207,7AB1
8,Wand Without Hope,Tracey Peake,Roob-Schoen,1881-12-26 00:00:00,0-9833-7301-9,349,2A,N206,6AB3
9,Spire Of Power,Tim Johnston,Glover-Windler,1829-10-02 00:00:00,0-2827-9962-1,530,2A,N201,1AB4


In [4]:
class Book():
    # date = dateutil.parser.parse(data['date'])
    title = ""
    auther = ""
    publisher = ""
    publicationYear = 0 
    ISBN = ""
    nPages = 0
    
    def __init__(self, title, auther="unknows", publisher="unknows", publicationYear=dateutil.parser.parse("0001-01-01"), ISBN="unknows", nPages=0):
        self.title = title
        self.auther = auther
        self.publisher = publisher
        self.publicationYear = publicationYear 
        self.ISBN = ISBN
        self.nPages = nPages
        
    def getISBN(self):
        return self.ISBN
    
    def __str__(self):
        return  "Book: \n" +\
                "Title: " + self.title + "\n" +\
                "Auther: " + self.auther + "\n" +\
                "Publisher: " + self.publisher + "\n" +\
                "Publication Year: " + self.publicationYear.isoformat() + "\n" +\
                "ISBN: " + self.ISBN + "\n" +\
                "Number of pages: " + str(self.nPages) + "\n" 
        

class Room():
    name = ""
    rows = dict()
    
    def __init__(self, name=""):
        self.name = name
    
    def getName(self):
        return self.name
    
    def addRow(self, row):
        self.rows[row.getName()] = row
        
    def removeRow(self, rowName):
        if (rowName in rows):
            del rows[rowName]
            
    def getRow(self, rowName):
        if (rowName in self.rows):
            return self.rows[rowName]
        else:
            row = Row(rowName)
            self.rows[rowName] = row
            return row
    
    def __str__(self):
        return list(self.rows.keys()).__str__() 

class Row():
    name = ""
    bookshelfs = dict()
    
    def __init__(self, name=""):
        self.name = name
    
    def getName(self):
        return self.name
    
    def addBookShelf(self, bookShelf):
        self.bookshelfs[bookShelf.getName()] = bookShelf
        
    def removeBookShelf(self, bookShelfName):
        if (bookShelfName in bookshelfs):
            del bookshelfs[bookShelfName]
    
    def getBookShelf(self, bookShelfName):
        if (bookShelfName in self.bookshelfs):
            return self.bookshelfs[bookShelfName]
        else:
            bookshelf = BookShelf(bookShelfName)
            self.bookshelfs[bookShelfName] = bookshelf
            return bookshelf
    
    def __str__(self):
        return list(self.bookshelfs.keys()).__str__() 
    
class BookShelf():
    name=""
    books = dict()
    
    def __init__(self, name=""):
        self.name = name
        
    def getName(self):
        return self.name
    
    def addBook(self, book):
        self.books[book.getISBN()] = book
        
    def removeBook(self, bookISBN):
        if (bookISBN in books):
            del books[bookISBN]
    
    def getBook(self, ISBN):
        if (ISBN in self.books):
            return self.books[ISBN]
        else:
             raise Exception("Book with ISBN [{}] DONT exists".format(ISBN))
    
    def __str__(self):
        return list(self.books.keys()).__str__()
    
class Library():
    name=""
    rooms = dict()
    
    def __init__(self, name=""):
        self.name = name
        
    def getName(self):
        return self.name
    
    def addRoom(self, room):
        self.rooms[room.getName()] = room
        
    def removeRoom(self, roomName):
        if (roomName in rooms):
            del rooms[roomName]
    
    def getRoom(self, roomName):
        if (roomName in self.rooms):
            return self.rooms[roomName]
        else:
            room = Room(roomName)
            self.rooms[roomName] = room
            return room
    
    
    def __str__(self):
        return list(self.rooms.keys()).__str__()

In [5]:
def n_books(df):
    '''
    return all books in the data base
    '''
    return df.shape[0]

def get_rooms(df):
    '''
    return all rooms in the data base
    '''
    return df['roomNumber'].drop_duplicates().values

def n_rooms(df):
    '''
    return number of rooms in the data base
    '''
    return df['roomNumber'].drop_duplicates().shape[0]

def get_rows(df):
    '''
    return all rows in the data base
    '''
    return df['rowNumber'].drop_duplicates().values

def n_rows(df):
    '''
    return number of rows in the data base
    '''
    return df['rowNumber'].drop_duplicates().shape[0]

def get_bookshelfs(df):
    '''
    return all bookshelf in the data base
    '''
    return df['bookShelfNumber'].drop_duplicates().values

def n_bookshelf(df):
    '''
    return number of bookshelf in the data base
    '''
    return df['bookShelfNumber'].drop_duplicates().shape[0]

def n_book_rooms(df):
    '''
    return number of books per room in the data base
    '''
    res = pd.crosstab(df['roomNumber'], df['title'])
    return res.sum(axis='columns')

def n_books_rows(df):
    '''
    return number of books per row in the data base
    '''
    res = pd.crosstab(df['rowNumber'], df['title'])
    return res.sum(axis='columns')

def n_books_bookshelfs(df):
    '''
    return number of books per row in the data base
    '''
    res = pd.crosstab(df['bookShelfNumber'], df['title'])
    return res.sum(axis='columns')

def get_count_rows_per_room(df):
    return pd.crosstab(df['roomNumber'], df['rowNumber'])

def get_count_bookshelf_per_row(df):
    return pd.crosstab(df['rowNumber'], df['bookShelfNumber'])

def get_count_book_per_bookshelf(df):
    return pd.crosstab(df['bookShelfNumber'], df['ISBN'])

def get_by_count_table(df, value):
    isshelf = df.loc[:,value] > 0
    value = df.loc[:,value][isshelf]
    return value.keys().array[0]

def get_bookshelf(df, ISBN):
    return get_by_count_table(df, ISBN)

def get_row(df, shelfbookNumber):
    return get_by_count_table(df, shelfbookNumber)

def get_room(df, rowNumber):
    return get_by_count_table(df, rowNumber)


In [6]:
if __name__ == "__main__":
    library = Library()
    
    table_rows_per_room = get_count_rows_per_room(df_books)
    table_bookshelf_per_row = get_count_bookshelf_per_row(df_books)
    table_book_per_bookshelf = get_count_book_per_bookshelf(df_books)
    
    rooms = get_rooms(df_books)
    rows = get_rows(df_books)
    bookshelf = get_bookshelfs(df_books)
    
    for i in range(n_books(df_books)):
        data = df_books.loc[i+1]
        book = Book(title=data['title'], auther=data['auther'], publisher=data['publisher'], publicationYear=dateutil.parser.parse(data['publicationYear']), ISBN=data['ISBN'], nPages=data['nPages'])
        room = library.getRoom(data['roomNumber'])
        row = room.getRow(data['rowNumber'])
        bookShelf = row.getBookShelf(data['bookShelfNumber'])
        bookShelf.addBook(book)

In [7]:
if __name__ == "__main__":
    print(library)
    ISBN = '0-4353-0966-8'
    bookshelfName = get_bookshelf(table_book_per_bookshelf,ISBN)
    rowName = get_row(table_bookshelf_per_row, bookshelfName)
    roomName = get_room(table_rows_per_room, rowName)
    room = library.getRoom(roomName)
    row = room.getRow(rowName)
    bookshelf = row.getBookShelf(bookshelfName)
    book = bookshelf.getBook(ISBN)
    print(book)

['1A', '2A', '3A', '4A', '5A']
Book: 
Title: Texts from Denmark
Auther: Brian Jensen
Publisher: Gyldendal
Publication Year: 1837-10-22T00:00:00
ISBN: 0-4353-0966-8
Number of pages: 915



In [9]:
d = dict()
d['a'] = 1
d['b'] = 2
d['c'] = 3
d['d'] = 4

In [11]:
for key in d.keys():
    print(d[key])

1
2
3
4
